# Multiclass Sentiment Analysis Using IMDB Reviews

**Authors:** Joseph Babel, Cameron Harte

**Description:**
We build several models using different classification algorithms in conjunction with n-gram and tf-idf features to find the best combination for classifying IMDB reviews into multiple sentiment classes.

**Task Distribution:**

Joseph Babel: Preprocess data, train and test SGD classifier, train and test SVM classifier, build confusion matrix with better prediction metrics

Cameron Harte: Build csv files for train and test data, build ngram models, train and test NB classifier, train and test ME classifier

**Required Modules:**
Python 3.8, scikit-learn 0.24.1, nltk 3.5

## Download (Only If Missing Stopwords File)

In [1]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Joey\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Imports

In [2]:
# FILE I/O
import csv  # generate csv files
import os  # operating system functions

# DATA PREPROCESSING
import re  # regex
from nltk.corpus import stopwords # stopwords
from nltk.stem.porter import * # stemmer

## Create CSV file
This will create both train and test csv files that include the raw text review with classification.

In [3]:
def create_csv_file(filename, train):
    header = ['row_number', 'text', 'classification']
    
    if train == True:
        path_to_mostly_neg = "Labeled-Data/Train/Mostly-Negative/"
        path_to_slightly_neg = "Labeled-Data/Train/Slightly-Negative/"
        path_to_neutral = "Labeled-Data/Train/Neutral/"
        path_to_slightly_pos = "Labeled-Data/Train/Slightly-Positive/"
        path_to_mostly_pos = "Labeled-Data/Train/Mostly-Positive/"
    else:
        path_to_mostly_neg = "Labeled-Data/Test/Mostly-Negative/"
        path_to_slightly_neg = "Labeled-Data/Test/Slightly-Negative/"
        path_to_neutral = "Labeled-Data/Test/Neutral/"
        path_to_slightly_pos = "Labeled-Data/Test/Slightly-Positive/"
        path_to_mostly_pos = "Labeled-Data/Test/Mostly-Positive/"
        
    count = 0
    
    with open(filename, "w", newline = '') as f1:
        writer = csv.writer(f1, delimiter = ',')
        writer.writerow(header)
        # add mostly negative reviews
        for f in os.listdir(path_to_mostly_neg):
            classification = 0
            if f.endswith(".txt"):
                open_file = open(path_to_mostly_neg+f, "r")
                data = open_file.read()
                writer.writerow([count, f'"{data}"', classification])
                count += 1
                open_file.close()
        # add slightly negative reviews
        for f in os.listdir(path_to_slightly_neg):
            classification = 1
            if f.endswith(".txt"):
                open_file = open(path_to_slightly_neg+f, "r")
                data = open_file.read()
                writer.writerow([count, f'"{data}"', classification])
                count += 1
                open_file.close()
        # add neutral reviews
        for f in os.listdir(path_to_neutral):
            classification = 2
            if f.endswith(".txt"):
                open_file = open(path_to_neutral+f, "r")
                data = open_file.read()
                writer.writerow([count, f'"{data}"', classification])
                count += 1
                open_file.close()
        # add slightly positive reviews
        for f in os.listdir(path_to_slightly_pos):
            classification = 3
            if f.endswith(".txt"):
                open_file = open(path_to_slightly_pos+f, "r")
                data = open_file.read()
                writer.writerow([count, f'"{data}"', classification])
                count += 1
                open_file.close()
        # add mostly positive reviews
        for f in os.listdir(path_to_mostly_pos):
            classification = 4
            if f.endswith(".txt"):
                open_file = open(path_to_mostly_pos+f, "r")
                data = open_file.read()
                writer.writerow([count, f'"{data}"', classification])
                count += 1
                open_file.close()

In [4]:
create_csv_file("imdb_train.csv", train = True)
create_csv_file("imdb_test.csv", train = False)

## Create List of Reviews and Classifications
review_text_train, review_text_test - contain review text in preparation for converting them into n-grams.

y_train, y_test - contain classification labels for both training and testing our model.

In [5]:
def create_reviews_list(filename):
    review_text = []
    classification = []
    with open(filename,'r') as csvfile:
            reader = csv.reader(csvfile, delimiter = ',')
            next(reader, None)
            for row in reader:
                review_text.append(row[1])
                classification.append(row[2])
    return review_text, classification

In [6]:
review_text_train, y_train = create_reviews_list("imdb_train.csv")
review_text_test, y_test = create_reviews_list("imdb_test.csv")

# convert classifications to list of integers
y_train = list(map(int, y_train))
y_test = list(map(int, y_test))

## Preprocess Data
1) Remove html tags

2) Remove special characters

3) Convert to lowercase

4) Remove stopwords

5) Apply stemming


In [7]:
def preprocess_data(review_text):
    stemmer = PorterStemmer()
    for index, row in enumerate(review_text):
        row = re.sub(r'<.*?>', '', row) # remove html tags
        row = re.sub(r'[^a-zA-Z. ]', '', row) # remove special characters
        row = row.lower() # convert to lowercase
        row = " ".join([stemmer.stem(word) for word in row.split() if word not in set(stopwords.words('english'))]) # remove stop words and apply stemming
        review_text[index] = row
    return review_text

In [8]:
review_text_train=preprocess_data(review_text_train)
review_text_test=preprocess_data(review_text_test)

## Convert Review Text Into N-Grams

In [1]:
def text_to_ngram(review_text_train, y_train, review_text_test, ngram_range, tfidf):
    # output of integers
    
    if tfidf == True:
        # use_idf when 'True' enables inverse-document-frequency re-weighting
        # ngram_range = ngram_range sets the lower and upper boundary of range of n-values
        tfidfvec = TfidVectorizer (use_idf = True, ngram_range = ngram_range)
        
        # training data learns vocabulary dictionary and returns document-term matrix 
        X_train = tfidfvec.fit_transform(review_text_train)
        
        # transforms the test data to document-term matrix
        X_test = tfidfvec.transform(review_text_test)
    else:
        cvec = CountVectorizer (analyzer = 'word', ngram_range = ngram_range)
        
        X_train = cvec.fit_transform(review_text_train)
        
        X_test = cvec.fit_transform(review_text_test)

    # Once SGDClassifier is imported this function call should work
    # 'hinge' gives a linear SVM
    #clf = sgd_classifier(loss = "hinge", penalty = "11")
    
    # 'fit' is used to make the linear model fit with SGD
    clf.fit(X_train, y_train)
    
    # this method predicts class labels for samples in x_test
    prediction = clf.predict(X_test)
    
    return prediction
                                    

## SGD Classifier (Stochastic Gradient Descent)

In [9]:
def sgd_classifier(review_text_train, y_train, review_text_test, ngram_range, tfidf):


SyntaxError: unexpected EOF while parsing (<ipython-input-9-d27ede24f2da>, line 2)

## SVM Classifier (Support Vector Machine)

In [ ]:
def svm_classifier(review_text_train, y_train, review_text_test, ngram_range, tfidf):
    

## NB Classifier (Naive Bayes)

In [ ]:
def nb_classifier(review_text_train, y_train, review_text_test, ngram_range, tfidf):
    
    #import for Naive Bayes from sklearn
    from sklearn.naive_bayes import MultinomialNB
    classifier = MultinomialNB()
    classifier.fit(X_train, y_train)
    
    y_prediction = classifier.predict(X_test)
    y_real_prediction = classifier.predict(X_test)
    
    # non-neg measure of the quality of estimator
    mean_square_error = ((y_prediction - y_test) ** 2).mean()
    
    # is a measure of the differences between values predicted
    root_mean_square_error = sqrt(mean_square_error)

## ME Classifier (Maximum Entropy)

In [ ]:
def me_classifier(review_text_train, y_train, review_text_test, ngram_range, tfidf
                  

## Predictions